In [16]:
import open3d as o3d
import numpy as np
import cv2, PIL
from PIL import Image
from cv2 import aruco
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib as mpl
import pandas as pd
import argparse
import imutils
import time
import sys
import csv
import datetime as dt

%matplotlib nbagg

In [7]:
class AzureKinect:
    def __init__(self):
        self.config = o3d.io.AzureKinectSensorConfig()
        self.device = 0
        self.align_depth_to_color = 1

    def start(self):
        self.sensor = o3d.io.AzureKinectSensor(self.config)
        if not self.sensor.connect(self.device):
            raise RuntimeError('Failed to connect to sensor')

    def frames(self):
            while 1:
                rgbd = self.sensor.capture_frame(self.align_depth_to_color)
                if rgbd is None:
                    continue
                color,depth = np.asarray(rgbd.color).astype(np.uint8),np.asarray(rgbd.depth).astype(np.float32) / 1000.0
                return color, depth
    #def stop(self):
     #   pause()

In [8]:
cam = AzureKinect()
cam.start()

[Open3D INFO] AzureKinectSensor::Connect
[Open3D INFO] sensor_index 0
[Open3D INFO] Serial number: 000276693112
[Open3D INFO] Firmware build: Rel
[Open3D INFO] > Color: 1.6.102
[Open3D INFO] > Depth: 1.6.75[6109.7]


In [9]:
ARUCO_DICT = {
	"DICT_4X4_50": cv2.aruco.DICT_4X4_50,
	"DICT_4X4_100": cv2.aruco.DICT_4X4_100,
	"DICT_4X4_250": cv2.aruco.DICT_4X4_250,
	"DICT_4X4_1000": cv2.aruco.DICT_4X4_1000,
	"DICT_5X5_50": cv2.aruco.DICT_5X5_50,
	"DICT_5X5_100": cv2.aruco.DICT_5X5_100,
	"DICT_5X5_250": cv2.aruco.DICT_5X5_250,
	"DICT_5X5_1000": cv2.aruco.DICT_5X5_1000,
	"DICT_6X6_50": cv2.aruco.DICT_6X6_50,
	"DICT_6X6_100": cv2.aruco.DICT_6X6_100,
	"DICT_6X6_250": cv2.aruco.DICT_6X6_250,
	"DICT_6X6_1000": cv2.aruco.DICT_6X6_1000,
	"DICT_7X7_50": cv2.aruco.DICT_7X7_50,
	"DICT_7X7_100": cv2.aruco.DICT_7X7_100,
	"DICT_7X7_250": cv2.aruco.DICT_7X7_250,
	"DICT_7X7_1000": cv2.aruco.DICT_7X7_1000,
	"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL,
	"DICT_APRILTAG_16h5": cv2.aruco.DICT_APRILTAG_16h5,
	"DICT_APRILTAG_25h9": cv2.aruco.DICT_APRILTAG_25h9,
	"DICT_APRILTAG_36h10": cv2.aruco.DICT_APRILTAG_36h10,
	"DICT_APRILTAG_36h11": cv2.aruco.DICT_APRILTAG_36h11
}

In [10]:
def marker_depth(id_centers, depth_array):
    output = {}
    height = depth_array.shape[0];
    for key in id_centers:
        center_x = round(id_centers[key][0]) 
        center_y = round(height - id_centers[key][1])
        output[key] = depth_array[center_y][center_x]*100 #cm
    return output

In [11]:
def depth_data():
    # Read depth from Kinect
    color, depth = cam.frames()
    intrinsic = o3d.camera.PinholeCameraIntrinsic(1280, 720, 601.1693115234375, 600.85931396484375, 637.83624267578125, 363.8018798828125)
    depth = o3d.geometry.Image(depth)
    img = o3d.geometry.Image(color)
    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(img, depth, depth_scale=1.0, convert_rgb_to_intensity=False)
    
    #to array
    color_array = np.asarray(rgbd.color)
    depth_array = np.asarray(rgbd.depth)
    color_img = Image.fromarray(color_array)
    depth_img = Image.fromarray(depth_array)
    color_img.save("color.jpg")
    
    #aruco detection
    frame = cv2.imread("./color.jpg")
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_50)
    parameters =  aruco.DetectorParameters_create()
    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
    frame_markers = aruco.drawDetectedMarkers(frame.copy(), corners, ids)
    
    #aruco visualization
    id_centers = {}
    if ids is not None:
        #plt.figure()
        #plt.imshow(frame_markers)
        for i in range(len(ids)):
            c = corners[i][0]
            #plt.plot([c[:, 0].mean()], [c[:, 1].mean()], "o", label = "id={0}".format(ids[i]))
            id_centers[ids[i][0]] = (c[:, 0].mean(), c[:, 1].mean())
        #plt.legend()
        #plt.show()
    depth_dict = marker_depth(id_centers, depth_array)
    if bool(depth_dict): 
        depth_avg = sum(depth_dict.values())/len(depth_dict.values())
    else:
        depth_avg = 0
        
    return depth_avg

In [12]:
depth_data()

15.300000458955765

In [23]:
def store_depth_data(freq, duration, filename):
    with open(filename, 'w', newline='') as output:
        time_end = time.time() + duration;
        while(time.time() < time_end):
            writer = csv.writer(output);
            t = time.localtime()
            current_time = time.strftime("%H:%M:%S", t)
            writer.writerow([current_time, depth_data()])
            #time.sleep(1/freq)

In [24]:
store_depth_data(44,10,  "test.csv")